# Train a Chess LLM on xLAN Datasets/Validate Model

In [ ]:
import torch
from transformers import AutoModelForCausalLM
from src.train import ChessTrainer
from peft import LoraConfig, get_peft_model

## Dataset

- use datasets in ./data/training to train your model
- use DownloadUpload.ipynb to Download a dataset from Hugging Face
- use DataPreProcessing.ipynb to create your own dataset

### CUDA
Check if CUDA is available.

In [ ]:
print(torch.cuda.is_available())

### Config

### Datasets

In [ ]:
dataset = "data/tokens/carlsen_max_768.tok"

## HYPERPARAMETERS
BATCH_SIZE = 16  # use the largest batch size that fits on your GPU
SAVE_STEPS = 2000  # how often to save a checkpoint
LOGGING_STEPS = 50  # how often to validate model and publish it to Weights & Biases
EPOCHS = 1  # how many epochs to train for - how many times to go through the dataset
LEARNING_RATE = 0.0001  # learning rate - how fast the model should learn
SKIP_VALIDATION = True  # skip validation and only save model checkpoints
WEIGHTS_AND_BIASES_ENABLED = True  # enable logging to Weights & Biases
USE_FP16 = True  # enable mixed precision training (GPU only)
XLANPLUS_ENABLED = True  # use xLanPlus tokenizer

PEFT_BASE_MODEL = "Leon-LLM/Leon-Chess-350k-BOS" # base model to be loaded (from hugging face) for fine-tuning
# PEFT_BASE_MODEL = "./Leon-LLM-Models/V45_GPT2_19k_20E_xLANplus/checkpoint-24000" # base model to be loaded (from disk) for fine-tuning

## CONFIG FOR FINE-TUNING
R = 128 # 8~26min, 16~26min, 32~26min
LORA_ALPHA = 32
LORA_DROPOUT = 0.1

peft_config = LoraConfig(  # https://huggingface.co/docs/peft/v0.10.0/en/package_reference/lora#peft.LoraConfig
    task_type="CAUSAL_LM", # This does not need to be changed for our use case
    inference_mode=False, # don't change this for training, only later for inference
    r=R,  # lower means faster training, but might underfit because of less complexity (experiments don't show that training time increases, which is rather weird)
    lora_alpha=LORA_ALPHA,  # scaling factor that adjusts the magnitude of the combined result (balances the pretrained model’s knowledge and the new task-specific adaptation)
    lora_dropout=LORA_DROPOUT,
    # use_rslora=True, # might work better (not tried yet)
)

peft_model = get_peft_model(AutoModelForCausalLM.from_pretrained(PEFT_BASE_MODEL), peft_config)

## MODEL NAME
model_name = f"TODO/set/model/name"

## SAVING MODEL
output_dir = f"TODO/set/output/dir/{model_name}"

print(f"Training {model_name} on {dataset} with batch size = {BATCH_SIZE} for {EPOCHS} epochs")
print(f"Saving model to {output_dir}")

### Train

In [ ]:
trainer = ChessTrainer(
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    epochs=EPOCHS,
    input_file=dataset,
    output_dir=output_dir,
    save_steps=SAVE_STEPS,
    logging_steps=LOGGING_STEPS,
    skip_validation=SKIP_VALIDATION,
    weight_and_biases=WEIGHTS_AND_BIASES_ENABLED,
    use_FP16=USE_FP16,
    notation="xLANplus" if XLANPLUS_ENABLED else "xLAN",
    peft=peft_model,
)

trainer.train()

# Push Model to Hugging Face

In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()
# model.push_to_hub("your-name/bigscience/mt0-large-lora")

# Load Model from Disk (fine-tuned LoRA model)

In [ ]:
# https://huggingface.co/docs/transformers/main/en/peft
model_dir = "./Leon-LLM-Models/V45_GPT2_19k_20E_xLANplus/checkpoint-24000"
peft_model_id = "./Leon-LLM-Models/V55_V45_GPT2_19k_20E_xLANplus_19k_1E_r128/V55_V45_GPT2_19k_20E_xLANplus_19k_1E_r128"
loaded_model = AutoModelForCausalLM.from_pretrained(model_dir)
loaded_model.load_adapter(peft_model_id)


In [ ]:
from src.generate_prediction import generate_prediction

loaded_model.inference_mode = True
loaded_model.eval()
input = "Pd2d4 Pd7d5 Pc2c4 Pc7c6"
loaded_model.to("cuda" if torch.cuda.is_available() else "cpu")
generate_prediction(input=input, num_tokens_to_generate=3, model=loaded_model, token_path="./src/tokenizer/xlan_tokens.json")[0]

# Inference (for fine-tuned LoRA model)

In [ ]:
# load model from hugging face

from src.generate_prediction import generate_prediction

peft_model = get_peft_model(AutoModelForCausalLM.from_pretrained(PEFT_BASE_MODEL), peft_config)
peft_model.inference_mode = True
peft_model.eval()
input = "Pd2d4 Pd7d5 Pc2c4 Pc7c6"
peft_model.to("cuda" if torch.cuda.is_available() else "cpu")
generate_prediction(input=input, num_tokens_to_generate=3, model=peft_model, token_path="./src/tokenizer/xlan_tokens.json")[0]